In [1]:
import json
import pandas as pd 

In [18]:
x = pd.read_json('products.json')
dict_cols = ['attributes','Name','Description','Text','Keywords','MainCharacteristics']
for colname in dict_cols:
  x[colname] = x[colname].apply(lambda x: {} if pd.isna(x) else x)
  _tempdf = pd.json_normalize(x[colname], errors='ignore')
  _tempdf = _tempdf.add_prefix(colname+'_')
  x = pd.concat([x, _tempdf], axis=1, sort=False)
x = x.drop(dict_cols, axis=1)

In [19]:
x.sample(10)

,_id,Alias,IsVisible,price,TaxClass,OrderUnit,MinOrder,Manufacturer,ManufacturerSKU,StockLevel,StockLevelAlert,SuperProduct,Image,IsPopular,Images,category,productType,EbayCategory,EbayDescription,attributes_Farbe,attributes_Größe,attributes_Qualität,attributes_Stärke,attributes_Breite,attributes_Material,attributes_Dicke,attributes_Produkt,attributes_Art,attributes_Körnung,attributes_Schlüsselblatt,attributes_Batteriefach,attributes_Laufzeit,attributes_Besonderheiten,attributes_Gehäusefarbe,attributes_Tintenfarbe,attributes_Nummer,attributes_Länge,attributes_Stempelkissen,attributes_Typ,attributes_SilcaRef,...,attributes_I,attributes_KeylineRef,attributes_L,attributes_M,attributes_P,Name_de,Name_cs,Name_da,Name_en,Name_nl,Name_ru,Name_fr,Description_de,Description_cs,Description_da,Description_en,Description_nl,Description_ru,Description_fr,Text_de,Text_cs,Text_da,Text_en,Text_nl,Text_ru,Text_fr,Keywords_de,Keywords_cs,Keywords_da,Keywords_en,Keywords_nl,Keywords_ru,Keywords_fr,MainCharacteristics_de,MainCharacteristics_cs,MainCharacteristics_da,MainCharacteristics_en,MainCharacteristics_nl,MainCharacteristics_ru,MainCharacteristics_fr
293,5c923071bd04513b5b521051,200079,1.0,None,normal,piece,1,,,9.0,30,,https://a-key-gmbh.com/uploads/5c924c578704fd4...,NaN,[https://a-key-gmbh.com/uploads/5cb4406e8c5203...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,GT7BP Silca - Fahrzeugschlüssel,,,GT7BP Silca - Fahrzeugschlüssel,,,,<p>GT7BP Silca</p>\n\n<p>für Auto</p>\n\n<p>&n...,,,<p>GT7BP Silca</p>\n\n<p>für Auto</p>\n\n<p>&n...,,,,<p>GT7BP Silca</p>\n\n<p>für Auto</p>\n\n<p>&n...,,,,,,,,,,,,,,,,,,,,
2261,5c923073bd04513b5b52186e,61001-0032,1.0,5.9,normal,piece,1,Silca,,0.0,None,61001,https://a-key-gmbh.com/uploads/5c924d968704fd4...,NaN,[https://a-key-gmbh.com/uploads/5cb4418f8c5203...,NaN,5c922fed1636e138dbad7216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Art. 4 SW Nr. 16,lang (100 mm),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Buntbartschlüssel Art. 4 SW-LG,,,Buntbartschlüssel Art. 4 SW-LG,,,,<p>(Schulte Schlagbaum) S2466 und S2005</p>,,,<p>(Schulte Schlagbaum) S2466 und S2005</p>,,,,<p>Buntbartschlüssel Art. 4SW und Art. 4 SW-LG...,,,,,,,,,,,,,,,,,,,,
3130,5d3998268e5adc44f32aa9bd,XDRN10EN,1.0,NaN,2,piece,1,NaN,NaN,0.0,5,NaN,https://a-key-gmbh.com/uploads/5d3998268e5adc4...,0.0,[https://a-key-gmbh.com/uploads/5d3998268e5adc...,5d6e7ed07fb44871f7da8bd3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,VVDI Version [CRSL],NaN,NaN,VVDI Version [CRSL],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1261,5c923072bd04513b5b521470,30091,1.0,None,normal,piece,1,,,NaN,None,,https://a-key-gmbh.com/uploads/5c924d058704fd4...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,AB11R Silca - Zylinderschlüsse,,,AB11R Silca - Zylinderschlüsse,,,,"<p><font size=""1"">AB11R</font> Silca</p>\n\n<p...",,,"<p><font size=""1"">AB11R</font> Silca</p>\n\n<p...",,,,"<p><font size=""1"">AB11R</font> Silca</p>\n\n<p...",,,,,,,AB11R Silca Zylinderschlüssel Börkey \...,,,,,,,,,,,,,
2664,5c923073bd04513b5b521b35,AS003081,1.0,11.9,normal,piece,1,,,40.0,3,,https://a-key-gmbh.com/uploads/5c924dfa8704fd4...,NaN,[https://a-key-gmbh.com/uploads/5cb442438c5203...,5c922fc971a7b33850f30f78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Funkschlüssel - Gehäuse kompatibel für Honda -...,,,Funkschlüssel - Gehäuse kompatibel für Honda -...,,,,"<p style=""text-align: center;""><strong><span s...",,,"<p style=""text-align: center;""><strong><span s...",,,,"<p style=""text-align: center;""><strong><span s...",,,,,,,,,,,,,,,,,,,,
2607,5c923073bd04513b5b5219c9,904913-0013,1.0,None,norma

In [23]:
!pip install clean-text[gpl]

     |████████████████████████████████| 71kB 3.2MB/s 
     |████████████████████████████████| 245kB 9.1MB/s 
  Created wheel for ftfy: filename=ftfy-5.8-cp36-none-any.whl size=45612 sha256=850737a8182383cac7c9a6828aae8d24b9a6ac0a7369ab35160775aa7878e91a
  Stored in directory: /root/.cache/pip/wheels/ba/c0/ef/f28c4da5ac84a4e06ac256ca9182fc34fa57fefffdbc68425b
Successfully built ftfy


In [24]:
import os
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import groupby
from bs4 import BeautifulSoup
from cleantext import clean
from collections import OrderedDict

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
stopwords = list(set(stopwords.words('english')))

ps = PorterStemmer()
nltk.download('wordnet') 
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [34]:
def clean_text(text):
  return clean(text,
                fix_unicode=True,               # fix various unicode errors
                to_ascii=True,                  # transliterate to closest ASCII representation
                lower=False,                     # lowercase text
                no_line_breaks=False,           # fully strip line breaks as opposed to only normalizing them
                no_urls=True,                  # replace all URLs with a special token
                no_emails=True,                # replace all email addresses with a special token
                no_phone_numbers=True,         # replace all phone numbers with a special token
                no_numbers=False,               # replace all numbers with a special token
                no_digits=False,                # replace all digits with a special token
                no_currency_symbols=True,      # replace all currency symbols with a special token
                no_punct=False,                 # fully remove punctuation
                replace_with_url=' url ',
                replace_with_email=' email ',
                replace_with_phone_number=' phone ',
                lang='en'
            )

In [37]:
def clean_l1(text):
  text = ' ' + text + ' '
  text = BeautifulSoup(text, "lxml").text
  text = re.sub(r'<.*?>', ' ', text)
  text = re.sub(r'\{[^{}]*\}', ' ', text)
  text = BeautifulSoup(text, "lxml").text
  text = clean_text(text)
  text = re.sub(r'[^A-Za-z0-9.,?\']', ' ', text)
  text = re.sub('[.]', ' . ', text)
  text = re.sub('[,]', ' , ', text)
  text = re.sub('[?]', ' ? ', text)
  text = ' '.join(text.split())
  text = re.sub(r'\b\w{1,1}\b', '', text)
  text = ' '.join([k for k,v in groupby(text.split())])  
  text = ' '.join(text.split())
  return text

In [42]:
x['Text_en'] = x['Text_en'].astype('str').apply(clean_l1)
x['Description_en'] = x['Description_en'].astype('str').apply(clean_l1)

In [44]:
x.sample(100).to_csv('sample_output.csv', index=False)

In [45]:
!mv /content/*.json '/content/drive/My Drive/Upwork/Temp'